In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
bank_customers = pd.read_csv("../input/bank-customer-churn-modeling/Churn_Modelling.csv")

In [ ]:
bank_customers.head(10)

## Exploratory Data Analysis (EDA)

In [ ]:
bank_customers.info()

In [ ]:
# Drop RowNumber and CustomerId because it won't be useful in predictive task
bank_customers.drop(['RowNumber', 'CustomerId', 'Surname'], axis=1, inplace=True)

**Categorical Features**

In [ ]:
# Convert object dtype into category
cat_features = ['Geography', 'Gender', 'HasCrCard', 'IsActiveMember', 'NumOfProducts', 'Tenure']
for colname in cat_features:
    bank_customers[colname] = bank_customers[colname].astype('category')

In [ ]:
bank_customers[cat_features].nunique()

In [ ]:
sns.countplot(x='Geography', data=bank_customers)

In [ ]:
sns.countplot(x='Gender', data=bank_customers)

In [ ]:
sns.countplot(x='HasCrCard', data=bank_customers)

In [ ]:
sns.countplot(x='IsActiveMember', data=bank_customers)

In [ ]:
sns.countplot(x='NumOfProducts', data=bank_customers)

In [ ]:
sns.countplot(x='Tenure', data=bank_customers, color='blue')

In [ ]:
bank_customers.groupby('Geography').mean()

In [ ]:
bank_customers.groupby('Gender').mean()

In [ ]:
bank_customers.groupby('HasCrCard').mean()

In [ ]:
bank_customers.groupby('IsActiveMember').mean()

In [ ]:
bank_customers.groupby("NumOfProducts").mean()

In [ ]:
bank_customers.groupby('Tenure').mean()

In [ ]:
bank_customers.groupby("Exited").mean()

In [ ]:
sns.countplot(x='NumOfProducts', hue='Exited', data=bank_customers)

In [ ]:
sns.countplot(x=bank_customers['IsActiveMember'], hue=bank_customers['Exited'])

In [ ]:
sns.countplot(x=bank_customers["Gender"], hue=bank_customers['Exited'])

In [ ]:
sns.boxplot(x=bank_customers['Geography'], y=bank_customers['Balance'])

In [ ]:
sns.boxplot(x=bank_customers['Geography'], y=bank_customers['Balance'], hue=bank_customers['Gender'])

Interesting Findings:

    1) 

**Numerical Features**

In [ ]:
bank_customers.columns
num_features = ['CreditScore', 'Age', 'Balance', 'EstimatedSalary']

In [ ]:
bank_customers[num_features].nunique()

In [ ]:
bank_customers['CreditScore'].hist(bins=30, grid=False)

In [ ]:
bank_customers[bank_customers['CreditScore'] == bank_customers['CreditScore'].max()]['Exited'].value_counts()

In [ ]:
bank_customers['Age'].hist(bins=30, grid=False)

In [ ]:
bank_customers['Balance'].hist(bins=30, grid=False)

In [ ]:
bank_customers[bank_customers['Balance'] == 0]['Exited'].value_counts()

In [ ]:
bank_customers['EstimatedSalary'].hist(bins=30, grid=False)

In [ ]:
#num_df = bank_customers[num_features + ['Exited']] 
sns.pairplot(bank_customers[num_features + ['Exited']] , hue='Exited')

Interesting finding: 

~ Features ~

- Out of all the customers who have the highest credit score, about 81% of them didn't exit.
- A third of the customers didn't deposit any money. Out of all these customers, about 86% of them exit.

~ Model ~

- Since most of the data overlaps, it's better we choose a non-linear classifier (decision tree, random forest, kNN etc.)

## Splitting into train/test/validation

In [ ]:
from sklearn.model_selection import train_test_split
X = bank_customers.drop(['Exited'], axis = 1)
y = bank_customers['Exited']
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_valid, y_train_valid, test_size = 0.25, random_state=42)

## Feature Engineering

In [ ]:
from sklearn.metrics import classification_report
from sklearn.dummy import DummyClassifier
target_names = ['Did not Exit', "Exited"]
dummy_clf = DummyClassifier(strategy="most_frequent")
dummy_clf.fit(X_train, y_train)
pred = dummy_clf.predict(X_train)
print(classification_report(y_train, pred))

In [ ]:
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
standardized_df = pd.DataFrame(ss.fit_transform(bank_customers[num_features]), columns = num_features)
standardized_df['Exited'] = bank_customers['Exited']

In [ ]:
from sklearn.preprocessing import LabelEncoder
le_geography = LabelEncoder()
le_gender = LabelEncoder()
le_HasCrCard = LabelEncoder()
le_IsActiveMember = LabelEncoder()
le_NumOfProducts = LabelEncoder()
le_Tenure = LabelEncoder()

le_df = pd.DataFrame()

le_df['Geography'] = le_geography.fit_transform(bank_customers['Geography'])
le_df['Gender'] = le_gender.fit_transform(bank_customers['Gender'])
le_df['HasCrCard'] = le_HasCrCard.fit_transform(bank_customers['HasCrCard'])
le_df['IsActiveMember'] = le_IsActiveMember.fit_transform(bank_customers['IsActiveMember'])
le_df['NumOfProducts'] = le_NumOfProducts.fit_transform(bank_customers['NumOfProducts'])
le_df['Tenure'] = le_Tenure.fit_transform(bank_customers['Tenure'])

le_df.head()

In [ ]:
model_df = pd.concat([standardized_df,le_df],axis=1)
model_df.head()

## Now that we have our Data in the desired form, we will be testing alot of different models:

In [ ]:
X = model_df.drop(['Exited'],axis=1)
y = model_df['Exited']
X_train, x_test, y_train, y_test = train_test_split(X,y)

### First, Let's do a DecisionTree:

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import mean_squared_error

dt_clf = DecisionTreeClassifier(max_depth=5,max_features='sqrt')

dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(x_test)
print(classification_report(y_test, pred))
fig, ax = plt.subplots(figsize=(20, 20))
tree.plot_tree(dt_clf.fit(X_train, y_train), ax=ax, filled=True)
plt.show()

### Now let's try a KNN Classifier:

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=3)
neigh.fit(X_train, y_train)
pred = neigh.predict(x_test)
print(classification_report(y_test, pred))

### Taking this further we develop a RandomForestClassifier:

In [ ]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=1800, min_samples_split=2, 
                             min_samples_leaf=1, max_features='auto', 
                             max_depth=10, bootstrap=True)
rf.fit(X_train,y_train)
pred = rf.predict(x_test)
print(classification_report(y_test, pred))

### Continuing with our Models, we are going to see how AdaBoost performs on our Data:

In [ ]:
from sklearn.ensemble import AdaBoostClassifier
ada_clf = AdaBoostClassifier(n_estimators=1800)
ada_clf.fit(X_train, y_train)
pred = ada_clf.predict(x_test)
print(classification_report(y_test, pred))

### We also want to see how a GaussionProcessClassifier performs:

In [ ]:
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
 
gpc = GaussianProcessClassifier().fit(X_train, y_train)

In [ ]:
gpc.score(X_train, y_train)

### Here we are seeing how a GaussianNB model performs on our Data:

In [ ]:
from sklearn.naive_bayes import GaussianNB
GNB_clf = GaussianNB()
GNB_clf.fit(X_train, y_train)
pred = GNB_clf.predict(x_test)
print(classification_report(y_test, pred))

### Along with these we are going to test a SVC model:

In [ ]:
from sklearn.svm import SVC
from sklearn.pipeline import make_pipeline
svc_clf = make_pipeline(StandardScaler(), SVC(gamma='auto'))
svc_clf.fit(X_train, y_train)
pred = svc_clf.predict(x_test)
print(classification_report(y_test, pred))

### Finally we are going to try a Neural Network model, MLPClassifier:

In [ ]:
from sklearn.neural_network import MLPClassifier
MLP_clf = MLPClassifier(max_iter=3000).fit(X_train, y_train)
pred = MLP_clf.predict(x_test)
print(classification_report(y_test, pred))

## Now that we have tried 8 different Models we evaluate the outputs.

## Ultimately we are going to be moving forward with RandomForest:

# Final Model:

## Hyperparameter tuning our Final Model and finalizing the tests

In [ ]:
rf = RandomForestClassifier(n_estimators=1800, min_samples_split=2, 
                             min_samples_leaf=1, max_features='auto', 
                             max_depth=10, bootstrap=True)
rf.fit(X_train,y_train)
pred = rf.predict(x_test)
print(classification_report(y_test, pred))

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform


n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]

max_features = ['auto', 'sqrt']

max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)

min_samples_split = [2, 5, 10]

min_samples_leaf = [1, 2, 4]

bootstrap = [True, False]

random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}


clf = RandomizedSearchCV(rf, random_grid)
search = clf.fit(X_train,y_train)
search.best_params_

In [ ]:
X_train, x_test, y_train, y_test = train_test_split(X,y,train_size=0.8)

In [ ]:
rf = RandomForestClassifier(n_estimators=2000, min_samples_split=2, 
                             min_samples_leaf=4, max_features='auto', 
                             max_depth=50, bootstrap=True)
rf.fit(X_train,y_train)
pred = rf.predict(x_test)
print(classification_report(y_test, pred))

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_test, pred)